In [1]:
import os
import json
import glob


In [2]:

class RawDataReader:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            self.data = json.load(file)

        self.id = self.data['ID']
        self.plan_details = self.extract_plan_details()

    def extract_plan_details(self):
        plan_details = []
        for plan in self.data['plans']:
            
            try:
                plan_name = plan['name']
                planning_exam_name = plan['planning_examination']['name']
                planningCT_filename = plan['planning_examination']['dose_DICOM_filename']
                    
                evaluation_exams = []
                for exam in plan['evaluation_examinations']:
                    if exam['completed'] == False:
                            pass
                    else:
                               
                        for opt_case in exam['optimization_cases']:
                            if opt_case.get('completed', True) and 'gradient_descent_result' in opt_case:
                                final_trans_coord = opt_case['gradient_descent_result'].get('final_translation_coordinate', {})
                                x = final_trans_coord.get('x', 0)
                                y = final_trans_coord.get('y', 0)
                                z = final_trans_coord.get('z', 0)
                                exam_detail = {
                                    'final_translation_coordinate': {'x': x, 'y': y, 'z': z},
                                    'repeatedCT_filename': opt_case['gradient_descent_result'].get('dose_DICOM_filename', '')
                                }
                                evaluation_exams.append(exam_detail)
                                break
                    
                plan_details.append({
                    'plan_name': plan_name,
                    'planning_exam_name': planning_exam_name,
                    'planningCT_filename': planningCT_filename,
                    'evaluation_examinations': evaluation_exams
                })

            except:
                plan_details.append({
                    'plan_name': None,
                    'planning_exam_name': None,
                    'planningCT_filename': None,
                    'evaluation_examinations': None
                })

        return plan_details

    def get_plan_details(self):
        return self.plan_details
    
# # Usage as an example
    
# file_path = '/home/shahpouriz/Data/DBP_oldDATA/nrrd/foton/results_DBP_HN025_20230705_220829.json'
file_path = '/home/shahpouriz/Data/DBP_oldDATA/results_DBP_HN045_20230701_174528.json'
treatment_data = RawDataReader (file_path)

# print(treatment_data.id)
plan_details = treatment_data.get_plan_details()
print(len(plan_details))
print(plan_details[0].get('plan_name'))
print(plan_details[0].get('planningCT_filename'))
print(plan_details[0].get('evaluation_examinations'))

1
AAPHH
DBP_HN045/AAPHH/rtdose_pCTp0_1.2.752.243.1.1.20230628045714840.1825.27442.dcm
[{'final_translation_coordinate': {'x': -0.13236685969027212, 'y': -0.02300256137457126, 'z': 0.11803708967199253}, 'repeatedCT_filename': 'DBP_HN045/AAPHH/rCTp2/optimization_case_0/rtdose_rCTp2_1.2.752.243.1.1.20230630221502174.7144.64777.dcm'}, {'final_translation_coordinate': {'x': -0.17628508997710285, 'y': -0.09899387964727158, 'z': 0.11579636220609804}, 'repeatedCT_filename': 'DBP_HN045/AAPHH/rCTp4/optimization_case_0/rtdose_rCTp4_1.2.752.243.1.1.20230630222756128.8420.84338.dcm'}, {'final_translation_coordinate': {'x': -0.2048481538405387, 'y': 0.17804733664382544, 'z': 0.045913224478790315}, 'repeatedCT_filename': 'DBP_HN045/AAPHH/rCTp7/optimization_case_0/rtdose_rCTp7_1.2.752.243.1.1.20230701164032666.4991.55163.dcm'}, {'final_translation_coordinate': {'x': -0.029906874041991993, 'y': -0.011326752934457285, 'z': 0.009613596718355266}, 'repeatedCT_filename': 'DBP_HN045/AAPHH/rCTp9/optimization

--------------------

In [3]:
class DirectoryProcessor:
    def __init__(self, directory_path):
        self.directory_path = directory_path

    def process_files(self):
        file_pattern = os.path.join(self.directory_path, 'results_DBP_HN*.json')
        json_files = glob.glob(file_pattern)
        results = []
        for file_path in json_files:
            reader = RawDataReader(file_path)
            file_results = {
                'id': reader.id,  # Ensure this is correctly capturing the 'ID' from each JSON
                'plan_details': reader.get_plan_details()
            }
            results.append(file_results)
        return results


# Usage
directory_path = '/home/shahpouriz/Data/DBP_oldDATA'
processor = DirectoryProcessor(directory_path)
all_results = processor.process_files()

for result in all_results:
    print(result)

{'id': 'DBP_HN014', 'plan_details': [{'plan_name': 'A1PHH', 'planning_exam_name': 'pCTp0', 'planningCT_filename': 'DBP_HN014/A1PHH/rtdose_pCTp0_1.2.752.243.1.1.20230622121511492.2918.80068.dcm', 'evaluation_examinations': [{'final_translation_coordinate': {'x': -0.011823806131906256, 'y': 0.007164642890292616, 'z': -0.0003346452569938929}, 'repeatedCT_filename': 'DBP_HN014/A1PHH/rCTp2/optimization_case_0/rtdose_rCTp2_1.2.752.243.1.1.20230622121808211.5511.94317.dcm'}, {'final_translation_coordinate': {'x': -0.06854204384431228, 'y': 0.10793481556814766, 'z': 0.10070348884390046}, 'repeatedCT_filename': 'DBP_HN014/A1PHH/rCTp7/optimization_case_0/rtdose_rCTp7_1.2.752.243.1.1.20230622133638535.3686.56661.dcm'}, {'final_translation_coordinate': {'x': -0.017402493119342277, 'y': 0.01798457435144707, 'z': 0.04108413205306627}, 'repeatedCT_filename': 'DBP_HN014/A1PHH/rCTp9/optimization_case_0/rtdose_rCTp9_1.2.752.243.1.1.20230622153408942.9936.99644.dcm'}, {'final_translation_coordinate': {'x

------------
# convert_dicom_to_nrrd

In [4]:
import os
import SimpleITK as sitk
import json  # Assuming you might need it for processing JSON data

class DICOMtoNRRDConverter:
    def __init__(self, directory_path, output_directory):
        self.directory_path = directory_path
        self.output_directory = output_directory
        self.directory_processor = DirectoryProcessor(directory_path)

    def convert_and_save(self):
        patient_data = self.directory_processor.process_files()

        for patient in patient_data:
            patient_id = patient['id']
            patient_output_path = os.path.join(self.output_directory, patient_id)

            if not os.path.exists(patient_output_path):
                os.makedirs(patient_output_path)

            print(len(patient['plan_details']))
            for plan_index, plan in enumerate(patient['plan_details']):
                
                plan_name = 'P'

                # Convert and update planning CT
                planning_ct_nrrd_name = self._generate_nrrd_filename(patient_id, plan_name, plan_index+1, "planningCT")
                nrrd_path = self._process_dose_file(plan['planningCT_filename'], patient_output_path, planning_ct_nrrd_name)

                plan['planningCT_filename'] = planning_ct_nrrd_name
                for exam_index, exam in enumerate(plan.get('evaluation_examinations', [])):
                    repeated_ct_nrrd_name = self._generate_nrrd_filename(patient_id, plan_name, plan_index+1, "repeatedCT", exam_index+1)

                    nrrd_path = self._process_dose_file(exam['repeatedCT_filename'], patient_output_path, repeated_ct_nrrd_name)
                    exam['repeatedCT_filename'] = repeated_ct_nrrd_name
        # Optional: Return the updated patient data for verification
        return patient_data
    
    
    def _generate_nrrd_filename(self, patient_id, plan_name, index, exam_type, e_index=None):
        filename = f"{patient_id}_{plan_name}{index}_{exam_type}"
        if e_index is not None:
            filename += f"{e_index}"
        filename += ".nrrd"
        return filename

    def _process_dose_file(self, dicom_name, folder_path, nrrd_filename):
        dicom_path = os.path.join(self.directory_path, dicom_name)
        nrrd_path = os.path.join(folder_path, nrrd_filename)
        
        try:
            dose_img = sitk.ReadImage(dicom_path)
            sitk.WriteImage(dose_img, nrrd_path)
            print(f"Successfully created: {nrrd_path}")
            return nrrd_path  # Return the new NRRD filename
        except Exception as e:
            print(f"Error processing file {dicom_path}: {e}")
            return None

# Assuming DirectoryProcessor is correctly implemented to parse input data

directory_path = '/home/shahpouriz/Data/DBP_oldDATA'
output_directory_proton = '/home/shahpouriz/Data/DBP_oldDATA/nrrd/test'
output_directory_foton = '/home/shahpouriz/Data/DBP_oldDATA/nrrd/foton'

converter = DICOMtoNRRDConverter(directory_path, output_directory_proton)
updated_patient_data = converter.convert_and_save()


2
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_planningCT.nrrd
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_repeatedCT1.nrrd
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_repeatedCT2.nrrd
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_repeatedCT3.nrrd
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_repeatedCT4.nrrd
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_repeatedCT5.nrrd
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_repeatedCT6.nrrd
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_repeatedCT7.nrrd
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/DBP_HN014/DBP_HN014_P1_repeatedCT8.nrrd
Successfully created: /home/shahpouriz/Data/D

------------
# convert_dicom_to_nrrd

In [5]:

# Save updated_patient_data to a JSON file
output_json_file = directory_path + '/nrrd/test/file_info.json' 
with open(output_json_file, 'w') as file:
    json.dump(updated_patient_data, file, indent=4)

print(f"Updated patient data saved to {output_json_file}")


Updated patient data saved to /home/shahpouriz/Data/DBP_oldDATA/nrrd/test/file_info.json
